In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Definition der Dataframes

In [3]:
df_walk = pd.read_json("normal_walk.json")

df_fall = pd.read_json("fall.json")

df_testFall = pd.read_json("test_fall.json")

In [5]:
df_walk.describe()

,time,seconds_elapsed,z,y,x,version,appVersion,yaw,qx,qz,roll,qw,qy,pitch
count,1.560600e+04,1.560600e+04,15605.000000,15605.000000,15605.000000,1.0,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0
mean,1.683269e+18,-6.919721e+02,0.051799,0.440262,0.023489,3.0,1.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,4.567039e+13,4.567039e+04,0.567014,1.009353,0.893979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.680417e+18,-2.852898e+06,-5.566208,-5.664675,-4.368199,3.0,1.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,1.683270e+18,1.962529e+01,-0.221366,-0.224200,-0.224045,3.0,1.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.683270e+18,3.925919e+01,0.022858,0.793248,0.062439,3.0,1.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,1.683270e+18,5.889272e+01,0.314255,1.077164,0.407181,3.0,1.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,1.683270e+18,7.852227e+01,4.551981,5.324697,4.855778,3.0,1.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_walk.columns

Index(['sensor', 'time', 'seconds_elapsed', 'z', 'y', 'x', 'version',
       'device name', 'recording time', 'platform', 'appVersion', 'device id',
       'sensors', 'sampleRateMs', 'yaw', 'qx', 'qz', 'roll', 'qw', 'qy',
       'pitch'],
      dtype='object')

In [9]:
#preprocessing data
df_walk['time'] = pd.to_datetime(df_walk['time'])
df_fall['time'] = pd.to_datetime(df_fall['time'])
df_testFall['time'] = pd.to_datetime(df_testFall['time'])

df_walk.head()

,sensor,time,seconds_elapsed,z,y,x,version,device name,recording time,platform,...,device id,sensors,sampleRateMs,yaw,qx,qz,roll,qw,qy,pitch
0,Accelerometer,2023-04-02 06:30:19.905724160,-2.852898e+06,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AccelerometerUncalibrated,2023-05-05 06:58:38.144353792,1.635376e-02,-0.770004,-0.392059,0.095901,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AccelerometerUncalibrated,2023-05-05 06:58:38.154418944,2.641895e-02,-0.801300,-0.401917,0.068222,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AccelerometerUncalibrated,2023-05-05 06:58:38.164484096,3.648413e-02,-0.832657,-0.412109,0.057846,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AccelerometerUncalibrated,2023-05-05 06:58:38.174548992,4.654907e-02,-0.863892,-0.420822,0.063507,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_run_acc = df_run[df_run['sensor'] == 'AccelerometerUncalibrated']
df_jump_acc = df_jump[df_jump['sensor'] == 'AccelerometerUncalibrated']